Bu kodun hazırlanmasında unsloth dökümantasyonundan yararlanılmıştır.

In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.3/347.3 kB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 116.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 2.0 MB/s eta 0:00:00:00:0100:0

Büyük Dil Modellerini olduğu gibi eğitmek için uzun zaman, fazla kaynak ve büyük veriye ihtiyaç vardır. Bu sorunun önüne geçmek için bir takım kütüphaneler ve yöntemler kullanılır. Kullanılan kütüphanelerden birisi de unsloth kütüphanesidir. Bu kütüphane istenilen açık kaynaklı büyük dil modellerini sisteme çeker ve çeşitli yöntemler sayesinde yüksek GB gereken veriyi küçülterek eğitimin kolay gerçekleştirilmesini ve çıktıların istenilen yerlerde saklanmasını sağlar. Unsloth kütüphanesinden kısaca bahsettiğimize göre yöntemlere değinelim.

Bahsedilen ve kodta geçen yöntemlerden birisi de 4bit kuantizasyon tekniğidir. Model ağırlıkları genellikle 32 bit float ya da 16 bit float olarak saklanır. Büyük modellerde bu ağırlıklar sistemde çok fazla yer kaplar. 4 bit kuantizasyon yöntemi ile ağırlıklar normalden 4 ile 8 kat daha az yer kaplar. Ancak saklanan verinin kesinliğinin (precision) düşük 16 ve 32 bitliğe göre daha düşük olmasından dolayı sonuçlar teorik olarak biraz düşük çıkabilir. Ancak modern tekniklerle (örn. Quantization-Aware Training, GPTQ) kayıp minimuma indirilebilir.

## Quantization-Aware Training (QAT)

Modeli FP32 veya FP16 olarak eğitirken aynı zamanda ağırlıkların kuantize edilmiş hallerini de simüle eder. Eğitim sırasında ağırlıklar hem float hem de kuantize edilmiş halde bulunur. Bu yöntemin amacı kuantizasyon sonrası performans kaybını önceden tahmin etmek ve minimize etmektir. Forward pass sırasında model, kuantize edilmiş ağırlıkları kullanır. Backpropagation sırasında gradient’ler float ağırlıklar üzerinden hesaplanır. Böylece model, kuantize edilmiş ağırlıkların getirdiği hataya "alışır" ve performans kaybı minimize edilir. Eğitim süresi artar, çünkü kuantizasyon simülasyonu ek hesaplama gerektirir.


## GPTQ (Gradient-based Post-Training Quantization)

Büyük modelleri eğittikten sonra kuantize eder. Gradient kullanarak 4-bit veya 3-bit kuantizasyon sırasında hatayı minimize eder. Model eğitildikten sonra ağırlıklar alınır. GPTQ, ağırlıkları grup veya blok bazlı olarak kuantize eder.
Kuantizasyon hatasını minimize etmek için local gradient’ler kullanılır (gradient descent ile optimize edilir, ama model yeniden eğitilmez). Sonuç olarak 4-bit veya 3-bit ağırlıklar, orijinal model performansına çok yakın bir doğrulukla elde edilir. Modeli yeniden eğitmek gerekmez. Post-training adımı, özellikle büyük modellerde zaman alabilir.

In [ ]:
# Model işlemleri için unsloth kütüphanesi kullanılır. 
from unsloth import FastLanguageModel
#Instruct modellerde bir chat template oluşturmak için bu kütüphane kullanılır.
from unsloth.chat_templates import get_chat_template
# Açık kaynak LLM'lerin çoğu torch kullanır.
import torch
# Huggingface datasetlerini kullanmak için datasets kütüphanesi kullanılır.
from datasets import load_dataset
# Model eğtimi için
from trl import SFTTrainer, SFTConfig

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/Qwen3-4B-Thinking-2507',
    load_in_4bit = True,'
    max_seq_length = 4096,
    dtype = None
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-10-23 07:19:37.925145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761203978.191483      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761203978.319079      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.8: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Rank parametresidir. =! 0 olmalıdır. 8, 16, 32, 64, 128 değerleri önerilir.
    # Bu değer arttıkça model doğruluğu artar ancak overfitting riski de artar.
    # Eğitilecek katmanlar
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    #lora parametreleri ve bias
    # özellikle lora da A B matrisinin katsayısı olarak lora_alpha / rank seçilir.
    lora_alpha = 32,
    lora_dropout = 0, 
    bias = "none",    
    # Uzun contextlerde True ya da unsloth ifadesi kullanılır.
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    #rank stabilized lora aktif mi pasif mi olacak onu belirler.
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth 2025.10.8 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [4]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [ ]:
ds = load_dataset("open-r1/OpenThoughts-114k-math")
ds

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00005.parquet:   0%|          | 0.00/199M [00:00<?, ?B/s]

data/train-00001-of-00005.parquet:   0%|          | 0.00/201M [00:00<?, ?B/s]

data/train-00002-of-00005.parquet:   0%|          | 0.00/203M [00:00<?, ?B/s]

data/train-00003-of-00005.parquet:   0%|          | 0.00/200M [00:00<?, ?B/s]

data/train-00004-of-00005.parquet:   0%|          | 0.00/176M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/89120 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'problem', 'solution', 'messages', 'system', 'conversations', 'generated_token_count', 'correct'],
        num_rows: 89120
    })
})

In [6]:
ds = ds['train'].select(range(10000))

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen3-thinking",
)

In [8]:
def format_prompts(batch):
    convs = batch["conversations"]

    texts = []
    for conv in convs:
        # "from" -> "role", "value" -> "content"
        chat = [
            {"role": msg["from"], "content": msg["value"]}
            for msg in conv
        ]

        # Şimdi Qwen3 chat template'ine uygun
        text = tokenizer.apply_chat_template(
            chat,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)

    return {"text": texts}


In [9]:
ds = ds.map(format_prompts, batched = True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [10]:
ds['text'][0]

'<|im_start|>user\nReturn your final response within \\boxed{}. Let \\( a, b, c \\) be positive real numbers. Prove that\n\n$$\n\\frac{1}{a(1+b)}+\\frac{1}{b(1+c)}+\\frac{1}{c(1+a)} \\geq \\frac{3}{1+abc},\n$$\n\nand that equality occurs if and only if \\( a = b = c = 1 \\).<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n<|begin_of_thought|>\n\nOkay, so I need to prove this inequality: \\(\\frac{1}{a(1+b)}+\\frac{1}{b(1+c)}+\\frac{1}{c(1+a)} \\geq \\frac{3}{1+abc}\\), where \\(a, b, c\\) are positive real numbers, and equality holds if and only if \\(a = b = c = 1\\). Hmm, let me try to think through this step by step.\n\nFirst, I remember that inequalities often involve techniques like AM-GM, Cauchy-Schwarz, or other classical inequalities. Maybe I can start by looking at each term on the left side and see if I can manipulate them to relate to the right side.\n\nThe left side has three terms: \\(\\frac{1}{a(1+b)}\\), \\(\\frac{1}{b(1+c)}\\), and \\(\\frac{1}{c(1+a)}\\). Each

Modeli eğitmeden önce bütün modeli eğitmek fazlaca yer ve zaman içereceğinden dolayı PEFT (Parameter Efficient Fine Tuning) tekniğini kullanırız. Bu bize belli katmanlar üzerinde model eğitimini sağlar ve bütün modeli eğitmektense modelin başlı kısımlarını eğitiriz. Ayrıca PEFT ile günümüzde sıkça kullanılan ve PEFT'in alt yöntemlerinden biri olan LORA (Low Rank Adaptation) tekniğini de sıkça kullanırız.

### PEFT’in alt yöntemleri:

 PEFT bir çatı kavramdır. Altında farklı teknikler vardır:

###
Teknik	        Açıklama

LoRA:	        En popüler yöntem. Ağırlıklara düşük-rank matris ekler.

Prefix Tuning:	Girdi dizisine “özel tokenlar” ekleyerek ayar yapar.

Adapter Tuning:	Her katmana küçük ek sinir ağı modülleri ekler.

Prompt Tuning:	Modeli yeniden eğitmeden, giriş metnine sanal promptlar ekler.
###

Bugün en çok kullanılanı LoRA’dır.

### LoRA (Low-Rank Adaptation)

LoRA, büyük matrisleri tamamen güncellemek yerine, onlara küçük, düşük-rank eklemeler yapar. Bir model katmanında normalde şöyle bir ağırlık vardır:

𝑦 = 𝑊 * 𝑥

Burada 𝑊 genelde çok büyük (örneğin 4096×4096 boyutunda) bir matristir. LoRA bunu şu şekilde değiştirir:

𝑦 =(𝑊+𝐴𝐵)𝑥

Burada 𝐴 ve 𝐵 küçük, düşük-rank matrislerdir. 𝑟 bu matrislerin boyutunu belirleyen parametre (örneğin 4, 8, 16). Eğitim sırasında 𝑊 sabit kalır (freeze). Sadece 𝐴 ve 𝐵 öğrenilir. Bu sayede parametre sayısı 100 kat azalabilir. Performans çoğu zaman aynı kalır veya çok az düşer. Bu sayede eğitilecek parametre sayısı azalır, bellekte tasarruf sağlanır ve eğitim hızlanır.

![image.png](attachment:c07ed10e-db72-4371-9bef-f25967c697c6.png)

### PEFT + LoRA birlikte nasıl çalışır?

PEFT, “hangi parametreleri eğitelim?” sorusuna genel cevaptır. LoRA, bu stratejilerden biridir (en verimlisi). PEFT önceden eğitilmiş katmanları alır ve belirli modüllere LoRA ekler. LoRA ise bu katmanları eğitir. 

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds,
    eval_dataset = None, 
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # 4 adımda gradyanlar toplansın
        warmup_steps = 5,
        #num_train_epochs = 3, # Full training için 1 ata
        #Modeli test ettiğimiz için max adım sayısı belirliyoruz.
        max_steps = 60,
        learning_rate = 2e-4, # Uzun traininglerde 2e-5 e düşür
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Wandb, tensorboard vs kullanmak istemiyorsak none
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [12]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [13]:
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.654700
2,0.544700
3,0.541600
4,0.512700
5,0.500800
6,0.490400
7,0.411900
8,0.471700
9,0.458800
10,0.452600


In [ ]:
messages = [
    {"role" : "user", "content" : "If f(x)=3x2−5x+2, find the value of f(4)."}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1000, 
    temperature = 0.7, top_p = 0.8, top_k = 20, 
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

</think>

</think>

<|begin_of_thought|>

Okay, so I need to find the value of f(4) for the function f(x) = 3x² - 5x + 2. Let me think about how to approach this. 

First, I remember that when you have a function like this, substituting the value of x into the equation will give you the result. So, f(4) means I should replace every x in the function with 4 and then perform the arithmetic operations. Let me write that down step by step to make sure I don't make a mistake.

Starting with the function: f(x) = 3x² - 5x + 2. 

Substituting x = 4 into the function:

f(4) = 3*(4)² - 5*(4) + 2.

Now, I need to calculate each term separately. Let's start with the first term, 3*(4)². 

First, calculate the exponent: 4 squared is 16. Then multiply by 3: 3*16 = 48. So the first term is 48.

Next term is -5*(4). Multiplying -5 by 4 gives me -20.

The last term is just +2, so that's straightforward.

Now, adding all the terms together: 48 (from the first term) minus 20 (from the second term) plus 2 

In [15]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [16]:
!zip -r lora_model.zip /kaggle/working/lora_model

  adding: kaggle/working/lora_model/ (stored 0%)
  adding: kaggle/working/lora_model/adapter_model.safetensors (deflated 8%)
  adding: kaggle/working/lora_model/adapter_config.json (deflated 56%)
  adding: kaggle/working/lora_model/merges.txt (deflated 57%)
  adding: kaggle/working/lora_model/chat_template.jinja (deflated 76%)
  adding: kaggle/working/lora_model/tokenizer_config.json (deflated 90%)
  adding: kaggle/working/lora_model/README.md (deflated 65%)
  adding: kaggle/working/lora_model/added_tokens.json (deflated 68%)
  adding: kaggle/working/lora_model/vocab.json (deflated 61%)
  adding: kaggle/working/lora_model/special_tokens_map.json (deflated 69%)
  adding: kaggle/working/lora_model/tokenizer.json (deflated 81%)
